<a href="https://colab.research.google.com/github/ru-corporate/boo/blob/master/boo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip uninstall boo -y
#!pip install --upgrade git+https://github.com/ru-corporate/boo.git@master

In [0]:
!pip install --upgrade boo

In [4]:
import boo
YEAR = 2018
boo.download(YEAR)
boo.build(YEAR)

Already downloaded: /root/.boo/raw2018.csv (1549.0M)
Use download(2018, force=True) to overwrite existing file.
Already built: /root/.boo/2018.csv (685.4M)
Use build(2018, force=True) to overwrite existing file.


In [0]:
!ls -s ~/.boo

total 2288064
 701880 2018.csv  1586184 raw2018.csv


In [5]:
from boo.dataframe.util import industry, inn, contains, sort
from boo.dataframe.filter import (large_companies, medium_companies,
                                  minimal_columns, is_alive, shorthand, 
                                  to_mln, to_bln)

try:
  root_df   
except NameError:
  root_df = boo.read_dataframe(YEAR)

try:   
  df
except NameError:
  df = shorthand(minimal_columns(large_companies(root_df)))


print("Исходное количество компаний ():               %7i" % root_df.shape[0])
print("Действующие крупные компании (после фильтров): %7i" % df.shape[0])

Исходное количество компаний ():               2282459
Действующие крупные компании (после фильтров):   22277


In [0]:
XL = "large_companies_2018.xlsx"
df.to_excel(XL)
from google.colab import files
files.download(XL)  # from colab to browser download

# Примеры выборок

In [6]:
# Случайные 5 компаний с активами более 100 млрд. руб.
df.query('ta>100').sample(5)

,region,ok1,ok2,title,ta,of,sales,p,cf
inn,,,,,,,,,
1655061006,16,64,99,ТЕЛЕКОМ-МЕНЕДЖМЕНТ,217.9,0.0,7.8,49.9,0.0
7701296415,77,64,99,ИНТЕР РАО КАПИТАЛ,139.9,0.0,2.7,13.3,0.3
5911066005,59,20,15,ЕВРОХИМ - УСОЛЬСКИЙ КАЛИЙНЫЙ КОМБИНАТ,122.8,115.3,3.1,-11.6,-0.7
3235002178,32,49,50,ТРАНСНЕФТЬ - ДРУЖБА,273.0,144.3,44.3,7.8,-2.3
7703270067,77,47,11,АШАН,139.6,58.6,270.7,3.6,-0.5


In [9]:
# Какие крупные компании не попадают в основные отрасли?
exclude = [64,66,46,6,24,49,19,20,35,47,45,29,41,42,61,68]
sort(df, "ta")[~df.ok1.isin(exclude)].head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,region,ok1,ok2,title,ta,of,sales,p,cf
inn,,,,,,,,,
7720437220,77,70,10,СВЕТЛАЯ ВЕЛИКАЯ РУСЬ,25710.1,0.0,0.0,0.0,0.0
7708514824,77,65,12,АГЕНТСТВО ПО СТРАХОВАНИЮ ВКЛАДОВ,3592.2,2.0,14.7,2.4,0.0
7705630445,77,70,10,РОСНЕФТЕГАЗ,3464.8,0.0,142.9,199.8,4.7
7708004767,77,70,10,"НЕФТЯНАЯ КОМПАНИЯ ""ЛУКОЙЛ""",2170.2,14.6,264.4,228.1,36.4
7709602713,77,9,10,ЯМАЛ СПГ,1858.4,1650.8,182.4,-206.7,-23.3


In [10]:
inn(df, 1660056570)

region                                                   16
ok1                                                      41
ok2                                                      20
title     ПРОИЗВОДСТВЕННО-СТРОИТЕЛЬНОЕ ОБЪЕДИНЕНИЕ "КАЗАНЬ"
ta                                                     54.7
of                                                        0
sales                                                   9.3
p                                                         0
cf                                                       -1
Name: 1660056570, dtype: object

In [11]:
for i in [41, 42]:
  print(i, boo.okved.name_v2(i))

41 Строительство зданий
42 Строительство инженерных сооружений


In [12]:
industry(df, 42).head(10)

,region,ok1,ok2,title,ta,of,sales,p,cf
inn,,,,,,,,,
7729588440,77,42,21,СТРОЙГАЗМОНТАЖ,217.8,1.7,304.1,13.8,3.2
7810170130,78,42,21,ГАЗПРОМ ИНВЕСТГАЗИФИКАЦИЯ,180.6,0.1,5.8,2.2,7.0
7701045732,77,42,11,МОСТОТРЕСТ,142.6,4.3,133.3,3.6,28.3
7714572888,77,42,21,СТРОЙТРАНСНЕФТЕГАЗ,129.0,8.3,186.5,9.3,-0.4
7810483334,78,42,21,ГАЗПРОМ ИНВЕСТ,118.3,7.6,67.9,0.4,0.0
7710884741,77,42,11,МАГИСТРАЛЬ СЕВЕРНОЙ СТОЛИЦЫ,93.8,0.5,6.9,-2.4,0.1
7703793405,77,42,11,МАГИСТРАЛЬ ДВУХ СТОЛИЦ,71.7,45.7,0.1,-0.1,1.9
7710068052,77,42,12,МОСКОВСКИЙ МЕТРОСТРОЙ,54.0,0.7,25.7,-1.6,1.6
7729747812,77,42,11,АВТОДОРОЖНАЯ СТРОИТЕЛЬНАЯ КОРПОРАЦИЯ,53.6,0.0,0.0,0.0,4.1


In [13]:
contains(df, "КРОКУС ИНТЕРНЭШНЛ")

,region,ok1,ok2,title,ta,of,sales,p,cf
inn,,,,,,,,,
7728115183,77,68,20,КРОКУС ИНТЕРНЭШНЛ,107.1,55.3,46.6,-5.4,-4.5


In [14]:
contains(df, "Стройтрансгаз")

,region,ok1,ok2,title,ta,of,sales,p,cf
inn,,,,,,,,,
5700000164,57,41,20,СТРОЙТРАНСГАЗ,45.7,0.3,26.9,0.4,4.9
7703813034,77,42,21,СТРОЙТРАНСГАЗ ТРУБОПРОВОДСТРОЙ,25.5,0.5,31.8,0.0,-0.0
2310048310,23,71,12,КРАСНОДАРСТРОЙТРАНСГАЗ,9.4,0.0,1.8,0.0,0.0
7714875963,77,41,20,СТРОЙТРАНСГАЗ,4.5,0.0,0.0,-0.1,0.0
7736608087,77,64,92,"ХОЛДИНГОВАЯ КОМПАНИЯ ""МОССТРОЙТРАНСГАЗ""",1.6,0.0,0.0,-0.0,0.0
7714880988,77,43,99,СТРОЙТРАНСГАЗ НЕФТЕГАЗМОНТАЖ,1.5,0.0,1.5,0.4,-0.0


In [15]:
contains(df, "Стройгазмонтаж")

,region,ok1,ok2,title,ta,of,sales,p,cf
inn,,,,,,,,,
7729588440,77,42,21,СТРОЙГАЗМОНТАЖ,217.8,1.7,304.1,13.8,3.2


# Компании по отраслям 

In [16]:
for i in boo.okved.all_codes_v2():
   x = industry(df.query('ta>5'), i).head(20)
   if not x.empty:
      print (i, boo.okved.name_v2(i))
      print (x.to_string(line_width=200))

1 Растениеводство и животноводство, охота и предоставление соответствующих услуг в этих областях
            region  ok1  ok2                               title     ta    of  sales     p   cf
inn                                                                                            
3252005997      32    1   50            БРЯНСКАЯ МЯСНАЯ КОМПАНИЯ  119.7  51.9   21.9 -14.2 -0.0
2328000083      23    1   50                        АГРОКОМПЛЕКС  118.8  35.1   50.0  -2.9  0.1
4609004168      46    1   46        АГРОПРОМКОМПЛЕКТАЦИЯ - КУРСК   52.6  36.5   13.7   3.0 -0.0
3110009570      31    1   46                СВИНОКОМПЛЕКС КОРОЧА   50.8   5.3   52.0  -0.8 -0.4
4812042756      48    1   46              ЧЕРКИЗОВО-СВИНОВОДСТВО   43.9  27.4   29.8   6.7  0.9
1207007950      12    1   47             ПТИЦЕФАБРИКА АКАШЕВСКАЯ   42.7  28.5   16.2  -0.4  0.3
4623004836      46    1   46                      МИРАТОРГ-КУРСК   40.0   0.1    0.2   0.2 -1.0
5828003909      58    1   47           

In [18]:
industry(df,56,10).head(10)

,region,ok1,ok2,title,ta,of,sales,p,cf
inn,,,,,,,,,
7710044140,77,56,10,МАКДОНАЛДС,27.0,19.8,69.3,6.0,-1.2
7719723690,77,56,10,БУРГЕР РУС,16.5,9.9,37.6,1.1,0.0
7709454818,77,56,10,АНВИАН,8.9,3.3,26.7,2.3,0.0
7722561551,77,56,10,ЯМ! РЕСТОРАНТС ИНТЕРНЭШНЛ РАША СИ АЙ ЭС,7.6,3.7,16.1,1.8,0.1
7703406825,77,56,10,РЕГИОНАЛЬНАЯ СЕТЬ ПРЕДПРИЯТИЙ ПИТАНИЯ,7.1,4.8,10.0,-0.2,-0.7
7802668116,78,56,10,СРП,7.1,2.0,1.5,0.1,1.5
7743242330,77,56,10,ИНТЕРНЭШНЛ РЕСТОРАНТ БРЭНДС,5.9,4.1,13.3,-0.4,0.1
7737115648,77,56,10,РЕСТОРАННАЯ ОБЪЕДИНЕННАЯ СЕТЬ И НОВЕЙШИЕ ТЕХНО...,5.6,1.1,6.6,0.0,0.0
7825335145,78,56,10,АМРЕСТ,5.0,2.0,10.2,-0.2,-0.0


In [22]:
contains(df, "бетон").head(5)

,region,ok1,ok2,title,ta,of,sales,p,cf
inn,,,,,,,,,
7714008255,77,71,12,ЗАВОД ЖЕЛЕЗОБЕТОННЫХ ИЗДЕЛИЙ № 23,15.8,0.0,0.0,0.0,-1.8
5009017294,50,23,61,ДОМОДЕДОВСКИЙ ЗАВОД ЖЕЛЕЗОБЕТОННЫХ ИЗДЕЛИЙ,12.3,1.4,0.0,-0.1,-0.0
2311011320,23,41,20,КРАСНОДАРСКИЙ ЗАВОД ЖЕЛЕЗОБЕТОННЫХ ИЗДЕЛИЙ №1,8.0,0.0,1.2,0.0,0.0
7814105213,78,23,99,АСФАЛЬТОБЕТОННЫЙ ЗАВОД-ВАД,7.4,1.0,13.4,0.7,-0.0
4703041174,47,49,41,ЦЕМЕНТНО-БЕТОННЫЕ ИЗДЕЛИЯ,7.3,4.1,5.5,0.1,-0.0


In [19]:
industry(df, 23, 51)

,region,ok1,ok2,title,ta,of,sales,p,cf
inn,,,,,,,,,
5022050558,50,23,51,ХОЛСИМ (РУС) СТРОИТЕЛЬНЫЕ МАТЕРИАЛЫ,41.1,26.3,15.1,-0.3,0.4
1322116731,13,23,51,МОРДОВЦЕМЕНТ,39.1,15.2,9.2,-1.6,0.0
2315020195,23,23,51,НОВОРОСЦЕМЕНТ,38.8,23.0,15.4,0.2,-0.1
4027077632,40,23,51,КАЛУЖСКИЙ ЦЕМЕНТНЫЙ ЗАВОД,29.9,26.5,0.0,-1.6,0.0
3202001147,32,23,51,МАЛЬЦОВСКИЙ ПОРТЛАНДЦЕМЕНТ,27.6,2.0,5.9,0.7,-0.0
6633001919,66,23,51,СУХОЛОЖСКЦЕМЕНТ,23.9,9.7,11.2,2.6,0.6
5036074848,50,23,51,ХАЙДЕЛЬБЕРГЦЕМЕНТ РУС,21.0,16.6,11.3,1.2,0.0
5607015014,56,23,51,ЮЖНО-УРАЛЬСКАЯ ГОРНО-ПЕРЕРАБАТЫВАЮЩАЯ КОМПАНИЯ,20.9,11.0,8.7,0.6,0.3
6441025673,64,23,51,ХОЛСИМ (РУС),19.0,17.6,5.5,-2.7,-0.4
